In [1]:
import pyspark
from pyspark.sql import SparkSession

#Starting Spark Session
spark = SparkSession.builder.getOrCreate()



In [3]:
#Getting Borough data
borough_data = spark.read.csv('data/taxi _zone_lookup.csv', header=True)
borough_data.show()


+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [4]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

#Get yellow taxis trip data from 2020
taxis_trips = spark.read.csv('data/tripdata/', header=True)

def get_month(value):
  return value.split("-")[1]
    
    
#Convert to a UDF Function (parameters: the function and its return type)
udf_set_month = F.udf(get_month, StringType())

#Add Month column to the dataset
trips = taxis_trips.withColumn("Month", udf_set_month("tpep_pickup_datetime"))

trips.show()
trips.select(trips.Month).show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Month|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----+
|       1| 2020-11-01 00:37:37|  2020-11-01 00:51:45|              1|         1.80|         1|                 N|         246|         137|           1|         10|    3|    0.5|       4.1|           0|   

In [5]:
#Join trip info with borough data
borough_trips = trips.join(borough_data, trips.PULocationID == borough_data.LocationID)
borough_trips.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----+----------+---------+--------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Month|LocationID|  Borough|                Zone|service_zone|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----+----------+---------+--------------------+------------+
|       1| 2020-11-01 00:37:37|  2020-11

# Período do ano em que os táxis são mais utilizados
## Critério: número de viagens

In [6]:
#Group the dataset by month, count the records and order by the count column
borough_trips.groupBy("Month").count().orderBy("count").show()

+-----+-------+
|Month|  count|
+-----+-------+
|   04| 237886|
|   05| 348370|
|   06| 549779|
|   07| 800430|
|   08|1007282|
|   09|1340892|
|   12|1462074|
|   11|1508900|
|   10|1681266|
|   03|3006942|
|   02|6299480|
|   01|6405198|
+-----+-------+



### Os táxis são mais utilizados em janeiro (6405198 viagens)

## Critério: quantidade de passageiros

In [7]:
#Casting passenger_count to Integer, since it is string
new_df = borough_trips.withColumn("passenger_count", borough_trips.passenger_count.cast(IntegerType()))

#Grouping by month and summing the number os passengers
new_df.groupBy("Month").agg(F.sum('passenger_count').alias('Passengers')).orderBy("Passengers").show()

+-----+----------+
|Month|Passengers|
+-----+----------+
|   04|    283231|
|   05|    380451|
|   06|    676814|
|   07|   1016698|
|   08|   1317999|
|   09|   1775148|
|   12|   1934072|
|   11|   1995476|
|   10|   2247303|
|   03|   4357543|
|   02|   9403624|
|   01|   9606865|
+-----+----------+



### O mês de janeiro é o que tem mais passageiros: 9606865